In [ ]:
import cv2
from sensor_msgs.msg import Image, CameraInfo
from realsense2_camera.msg import Extrinsics

from cv_bridge import CvBridge
import rospy

import pyrealsense2
import numpy as np
from matplotlib import pyplot as plt

import open3d as o3d
from scipy import spatial
from scipy.spatial.transform import Rotation as R
import math
import tf2_ros, tf
import geometry_msgs.msg
import PyKDL
import time

import sys
import moveit_commander
import moveit_msgs.msg
from moveit_commander.conversions import pose_to_list

import subprocess
from ast import literal_eval
import copy
from PIL import Image as PIL_img
from io import BytesIO
from configparser import ConfigParser
import glob

image = None

rospy.init_node("Cognitive_Motion", anonymous=True)

moveit_commander.roscpp_initialize(sys.argv)
robot = moveit_commander.RobotCommander()
scene = moveit_commander.PlanningSceneInterface()
group_name = "panda_manipulator"  #"panda_arm"
move_group = moveit_commander.MoveGroupCommander(group_name) #we'll pass it on while calling functions


tfbuffer = tf2_ros.Buffer()
listener = tf2_ros.TransformListener(tfbuffer)
br = tf2_ros.TransformBroadcaster()
t = geometry_msgs.msg.TransformStamped()

static_br = tf2_ros.StaticTransformBroadcaster()
static_t = geometry_msgs.msg.TransformStamped()

bridge = CvBridge()
loop_rate = rospy.Rate(0.5) # Node cycle rate (in Hz).
np.set_printoptions(suppress=True)

config = ConfigParser() #For config.ini

print("loaded")

In [ ]:
def grab_frame():
    
    frame_color=rospy.wait_for_message('/camera/color/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_color = bridge.imgmsg_to_cv2(frame_color, desired_encoding='rgb8')
    
    frame_depth = rospy.wait_for_message('/camera/depth/image_raw', Image, timeout=None) #wait_for_message(topic, topic_type, timeout=None): 
    cv_image_depth = bridge.imgmsg_to_cv2(frame_depth)
    
    return cv_image_color, cv_image_depth

def get_cam_param(depth_to_color=False):
    frame_depth_info=rospy.wait_for_message('/camera/depth/camera_info', CameraInfo, timeout=None)
    
    w = frame_depth_info.width
    h = frame_depth_info.height
    fx = frame_depth_info.K[0]
    fy = frame_depth_info.K[4]
    cx = frame_depth_info.K[2]
    cy = frame_depth_info.K[5]
    
    if depth_to_color:
        frame_depth_extrinsic=rospy.wait_for_message('/camera/extrinsics/depth_to_color', Extrinsics, timeout=None)
        return w,h,fx,fy,cx,cy,frame_depth_extrinsic.translation,frame_depth_extrinsic.rotation
    else:
        rot = [1,0,0, 0,1,0, 0,0,1]
        trans = [0, 0, 0]
        return w,h,fx,fy,cx,cy, trans, rot

def Generate_PointCloud(color_frame,depth_frame,trim_base,from_depth=False,depth_scale = 1000.0, depth_trunc=1.0,align=False,Dbug=False,zoom_def = 1.34): #Truncate distances at 1Meter default
    
    w,h,fx,fy,cx,cy, trans, rot = get_cam_param(depth_to_color=align)   

    cam = o3d.camera.PinholeCameraParameters()
    cam.intrinsic = o3d.camera.PinholeCameraIntrinsic(w, h, fx,fy, cx, cy)

    cam.extrinsic = np.array([[rot[0], rot[1], rot[2], trans[0]], 
                              [rot[3], rot[4], rot[5], trans[1]], 
                              [rot[6], rot[7], rot[8], trans[2]], 
                              [0.    , 0.    , 0.    , 1.      ]])
    

    color_raw = o3d.geometry.Image(np.asarray(color_frame))
    depth_raw = o3d.geometry.Image(np.asarray(depth_frame.astype(np.uint16)))
    
    if from_depth==False:
        rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity=False)
        pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic, cam.extrinsic)
        #CROP TO FILTER OUT ROBOT'S SHADOW ADJUST OFFSET ACCORDINGLY
        PC_BBOX = pcd.get_axis_aligned_bounding_box()
        minB_X = PC_BBOX.min_bound[0]
        maxB_X = PC_BBOX.max_bound[0]
        minB_Y = PC_BBOX.min_bound[1]
        maxB_Y = PC_BBOX.max_bound[1]
        minB_Z = PC_BBOX.min_bound[2]
        maxB_Z = PC_BBOX.max_bound[2]
        
        #RAW Point cloud from depth
        if Dbug==True:
            mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
            o3d.visualization.draw_geometries([pcd,mesh], window_name='01_RAW Pointcloud: from Simulated Camera', point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)
        
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(minB_X, minB_Y, minB_Z), 
                                                   max_bound=(maxB_X, maxB_Y, maxB_Z-trim_base)) #filter ground part 
        pcd = pcd.crop(bbox)
        #pcd = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd_image, cam.intrinsic)
        

        
        
    else:
        pcd =  o3d.geometry.PointCloud.create_from_depth_image(depth_raw, cam.intrinsic, cam.extrinsic, 
                                                               depth_scale, depth_trunc,
                                                               stride=1, project_valid_depth_only=True)
        #RAW Point cloud from depth
        if Dbug==True:
            mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
            o3d.visualization.draw_geometries([pcd,mesh], window_name='01_RAW Pointcloud: from Real Camera', point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)

    #pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]]) # Flip it, otherwise the pointcloud will be upside down.


    return pcd


def tst_dataset(color_frame,depth_frame):
    color_raw = o3d.geometry.Image(np.asarray(color_frame))
    depth_raw = o3d.geometry.Image(np.asarray(depth_frame.astype(np.uint16)) )
    rgbd_image = o3d.geometry.RGBDImage.create_from_color_and_depth(color_raw, depth_raw, convert_rgb_to_intensity=False)
    pcd = visualize_rgbd(rgbd_image,align=False)
    return pcd


def box_pos(x_coord, y_coord, width, height, centered=0):
    if centered == 0:
        start_point = (x_coord, y_coord) # represents the top left corner of rectangle
        end_point = (x_coord+width-1, y_coord+height-1)  # represents the bottom right corner of rectangle
    elif centered == 1:
        new_x = x_coord - (np.floor(width/2)-1).astype(int)
        new_y = y_coord - (np.floor(height/2)-1).astype(int)
        start_point = (new_x, new_y) 
        end_point = (new_x+width, new_y+height)
    return start_point, end_point


def fetch_transform(tfbuffer,frame1,frame2,quat=0):
    flag = 0
    while flag==0:
        try:
            trans = tfbuffer.lookup_transform(frame1, frame2, rospy.Time(),rospy.Duration(8.0))
            #print (trans)
            trans = trans.transform  #save translation and rotation
            #rot = PyKDL.Rotation.Quaternion(* [ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w] )
            #ypr = [ i  / np.pi * 180 for i in rot.GetEulerZYX() ]
            #print(ypr[2],ypr[1],ypr[0])
            
            rot = R.from_quat([ trans.rotation.x,trans.rotation.y,trans.rotation.z,trans.rotation.w]) #creates rotation matrix
            rpy = rot.as_euler('XYZ',degrees=True) #extrinsic
            break
        except (tf2_ros.LookupException, tf2_ros.ConnectivityException, tf2_ros.ExtrapolationException) as e:
            #print ("Fail", e)
            continue
    if quat==0:
        return trans.translation.x, trans.translation.y, trans.translation.z, rpy[0],rpy[1],rpy[2] #ypr[2], ypr[1], ypr[0]
    elif quat==1:
        return trans.translation.x, trans.translation.y, trans.translation.z, trans.rotation.x, trans.rotation.y, trans.rotation.z, trans.rotation.w
    


def Axis_angle_to_Quat(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    qx = x * np.sin(angle/2)
    qy = y * np.sin(angle/2)
    qz = z * np.sin(angle/2)
    qw = np.cos(angle/2)
    
    return qx,qy,qz,qw

def Axis_angle_to_Euler(vector,angle):
    x=vector[0]
    y=vector[1]
    z=vector[2]
    s=np.sin(angle)
    c=np.cos(angle)
    t=1-c
    
    if ((x*y*t + z*s) > 0.998):  #north pole singularity detected
        
        heading = 2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = np.pi/2
        bank = 0
        return heading, attitude, bank
    
    elif ((x*y*t + z*s) < -0.998):
        
        heading = -2 * np.arctan2(x*np.sin(angle/2), np.cos(angle/2))
        attitude = -np.pi/2
        bank = 0
        return heading, attitude, bank
    
    heading = np.arctan2(y * s- x * z * t , 1 - (y*y+ z*z ) * t)
    attitude = np.arcsin(x * y * t + z * s)
    bank = np.arctan2(x * s - y * z * t , 1 - (x*x + z*z) * t)
    
    return bank, heading, attitude


def getRotation(v1):
    if np.all(v1==[0., 0., 1.]): v1 = [0, 0.000001, 0.999999]
    vec_x = [1.,0.,0.] 
    vec_y = [0.,1.,0.]
    vec_z = [0.,0.,1.] #>>>>>>>>>>>>>>>>>>>>> change these to -1 to flip again
    
    vec1 = v1 / np.linalg.norm(v1)

    #vector_x = np.cross(vec1, vec_x)/np.linalg.norm(np.cross(vec1, vec_x))
    angle_x = (math.acos(np.dot(vec1, vec_x)))

    #vector_y = np.cross(vec1, vec_y)/np.linalg.norm(np.cross(vec1, vec_y))
    angle_y = (math.acos(np.dot(vec1, vec_y)))

    vector_z = np.cross(vec1, vec_z)/np.linalg.norm(np.cross(vec1, vec_z))
    angle_z = -(math.acos(np.dot(vec1, vec_z)))  #<<<Changed to adapt

    #Rotation = filtered_Pt_Cloud.get_rotation_matrix_from_axis_angle(angle*vector) #alternative Open3D lib.
    Rotation = R.from_rotvec(angle_z*vector_z)
    
    return Rotation, angle_x, angle_y, angle_z


def Bounds_gen(minB, maxB, spacing):
    ctr = 0
    bounds = np.array([[0.,0.]])
    CurrB = minB

    while CurrB < maxB-0.5*spacing: #from left most to right most

        LowerB = minB + ctr * spacing #if we shift the X or Y coordinates in multiples of spacing, we should get different lines.
        if ctr == 0:
            CurrB = LowerB + spacing/2 #lower + spacing/2 only for first condition
        else:
            CurrB = LowerB + spacing #lower + spacing
        bounds = np.append(bounds,[[LowerB,CurrB]], axis=0)
        #print("CurrB:",CurrB,"MaxB:",maxB,"diff:",maxB-CurrB)
        ctr+= 1
    bounds = np.delete(bounds, 0, axis=0)
    return bounds

def cropped_PC(original_PC, spacing, Cluster_trim, X=0, idx = 0, centered = True, resample = True):
    PC_BBOX = original_PC.get_axis_aligned_bounding_box()
    minB_X = PC_BBOX.min_bound[0]
    maxB_X = PC_BBOX.max_bound[0]
    minB_Y = PC_BBOX.min_bound[1]
    maxB_Y = PC_BBOX.max_bound[1]
    minB_Z = PC_BBOX.min_bound[2]
    maxB_Z = PC_BBOX.max_bound[2]
    nor = np.array(original_PC.normals)
    pts = np.array(original_PC.points)

    Xs,Ys = get_XY_angles_from_PC(nor)
    X_dev = np.std(Xs)
    Y_dev = np.std(Ys)
    print("X Standard Dev:", X_dev, "    Y Standard Dev:", Y_dev)
    
    if(X > 1):
        if(X==2): #Manual override at 2
            X = 0  
            print("Manual Override! taking curve around World Y-axis","X=", X)
        elif(X==3):  #Manual override at 3
            X = 1
            print("Manual Override! taking curve around World X-axis","X=", X)
        else: X = 1
    else:
        
        if (X_dev <= Y_dev): 
            X = 1
            print("Object's curve around World X-axis","X=", X)
        elif (X_dev > Y_dev): 
            X = 0
            print("Object's curve around World Y-axis","X=", X)

    if X == 1: #X sided sweep crop
        bounds = Bounds_gen(minB_X, maxB_X, spacing)
        print("Total bounds:",len(bounds))
        if centered == True:
            idx = int(np.floor(len(bounds)/2))
        else:
            if idx > len(bounds)-1:
                print("Warning! Index value out of bounds, setting to max value")
                idx = -1
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(bounds[idx][0], minB_Y+Cluster_trim, minB_Z), 
                                                   max_bound=(bounds[idx][1], maxB_Y, maxB_Z)) #trim edges
        result = original_PC.crop(bbox)
        res_pts = np.array(result.points)
        res_nor = np.array(result.normals)

        if resample == True:
            min_idx = np.argmin( np.std(res_pts, axis=0) )
            res_pts[:, min_idx] = np.mean(res_pts,axis=0)[min_idx]  #replace values of axis (x,y or z)
                                                                    #that has min std_dev by its mean.
            result.points = o3d.utility.Vector3dVector(res_pts)
            result = result.voxel_down_sample(voxel_size=spacing)
        
        res_pts = np.array(result.points)
        res_nor = np.array(result.normals)

        ind = np.argsort(res_pts[:, 1]) #Sort Y coordinates from lowest to highest, X values are almost constant
        res_pts = res_pts[ind]  #no longer required to change values to negative 
        res_nor = res_nor[ind]

    elif X == 0: #Y sided sweep crop
        bounds = Bounds_gen(minB_Y, maxB_Y, spacing)
        print("Total bounds:",len(bounds))
        if centered == True:
            idx = int(np.floor(len(bounds)/2))
        else:
            if idx > len(bounds)-1:
                print("Warning! Index value out of bounds, setting to max value")
                idx = -1
        bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(minB_X+Cluster_trim, bounds[idx][0], minB_Z), 
                                                   max_bound=(maxB_X, bounds[idx][1], maxB_Z))
        print(bbox)

        result = original_PC.crop(bbox)
        res_pts = np.array(result.points)
        res_nor = np.array(result.normals)
        
        if resample == True:
            min_idx = np.argmin( np.std(res_pts, axis=0) )
            res_pts[:, min_idx] = np.mean(res_pts,axis=0)[min_idx]  #replace values of axis (x,y or z)
                                                                    #that has min std_dev by its mean.
            result.points = o3d.utility.Vector3dVector(res_pts)
            result = result.voxel_down_sample(voxel_size=spacing)
        
        res_pts = np.array(result.points)
        res_nor = np.array(result.normals)

        ind = np.argsort(res_pts[:, 0]) #Sort X coordinates from lowest to highest, Y values are almost constant
        res_pts = res_pts[ind]
        #res_pts[:,2] *=-1  #Change z values to negative
        res_nor = res_nor[ind]

    sorted_pointcloud = o3d.geometry.PointCloud()
    sorted_pointcloud.points = o3d.utility.Vector3dVector(res_pts)
    sorted_pointcloud.normals = o3d.utility.Vector3dVector(res_nor)

    return sorted_pointcloud


def generate_coordinates(point_cloud,silent=True):
    Coordinates = np.array([[0.,0.,0.,0.,0.,0.,0.]])
    world_Coordinates = np.array([[0.,0.,0.,0.,0.,0.,0.]])
    point_cloud_pts = np.array(point_cloud.points)
    point_cloud_nor = np.array(point_cloud.normals)
    
    for index in range (len(point_cloud_pts)):

        rotat, angle_x, angle_y, angle_z = getRotation(-point_cloud_nor[index])  #<<<changed here pts -ve

        a1 = rotat.as_matrix()[0][0]
        a2 = rotat.as_matrix()[0][1]
        a3 = rotat.as_matrix()[0][2]
        b1 = rotat.as_matrix()[1][0]
        b2 = rotat.as_matrix()[1][1]
        b3 = rotat.as_matrix()[1][2]
        c1 = rotat.as_matrix()[2][0]
        c2 = rotat.as_matrix()[2][1]
        c3 = rotat.as_matrix()[2][2]

        KDL_original_plane_frame = PyKDL.Frame(PyKDL.Rotation(a1,a2,a3, b1,b2,b3, c1,c2,c3),
                                               PyKDL.Vector(point_cloud_pts[index][0],
                                                            point_cloud_pts[index][1],
                                                             point_cloud_pts[index][2]))

        KDL_flip_frame = PyKDL.Frame(PyKDL.Rotation.RPY(0, np.pi, 0), PyKDL.Vector(0,0,0)) #mirror plane frame to match camera frame
        KDL_final_frame = KDL_original_plane_frame * KDL_flip_frame
        KDL_trans = KDL_final_frame.p
        KDL_ROT_quat = KDL_final_frame.M.GetQuaternion() 

        Coordinates = np.append(Coordinates,[[KDL_trans[0] ,KDL_trans[1],KDL_trans[2] ,KDL_ROT_quat[0] 
                                     ,KDL_ROT_quat[1] ,KDL_ROT_quat[2],KDL_ROT_quat[3]]],axis=0)

        Publish_coordinates([[KDL_trans[0] ,KDL_trans[1],KDL_trans[2] ,KDL_ROT_quat[0] 
                            ,KDL_ROT_quat[1] ,KDL_ROT_quat[2],KDL_ROT_quat[3]]], 
                            'camera_depth_optical_frame', 'plane', static = True)

        transform_plane = fetch_transform(tfbuffer,'world', 'plane_static_0',quat=1)

        world_Coordinates = np.append(world_Coordinates,[[transform_plane[0] ,transform_plane[1]
                                            ,transform_plane[2] ,transform_plane[3] ,transform_plane[4] 
                                             ,transform_plane[5] ,transform_plane[6]]],axis=0)
        #time.sleep(0.03)
    Coordinates = np.delete(Coordinates, 0, axis=0)
    world_Coordinates2 = np.delete(world_Coordinates, 0, axis=0)
    
    ctr = 0
    vote=0
    diff_neg_x = np.sum(np.diff(world_Coordinates2[:,0:1],axis=0 ) < 0)
    diff_pos_x = np.sum(np.diff(world_Coordinates2[:,0:1],axis=0 ) >= 0)
    diff_neg_y = np.sum(np.diff(world_Coordinates2[:,1:2],axis=0 ) < 0)
    diff_pos_y = np.sum(np.diff(world_Coordinates2[:,1:2],axis=0 ) >= 0)
    diff_neg_z = np.sum(np.diff(world_Coordinates2[:,2:3],axis=0 ) < 0)
    diff_pos_z = np.sum(np.diff(world_Coordinates2[:,2:3],axis=0 ) >= 0)
    #print(diff_neg_x, diff_pos_x, diff_neg_y, diff_pos_y, diff_neg_z, diff_pos_z)
    while True: #check for anomalies in data
        idx = int(len(world_Coordinates2)/2)
        #Series is decresing and current number is bigger than middle one, no need to move or reverse.
        if (diff_neg_x>diff_pos_x and world_Coordinates2[0,0:1] >= world_Coordinates2[idx,0:1]) or (
            diff_neg_x<diff_pos_x and world_Coordinates2[0,0:1] < world_Coordinates2[idx,0:1]):
            #print("no need to move")
            vote += 0
        elif (diff_neg_x>diff_pos_x and world_Coordinates2[0,0:1] < world_Coordinates2[idx,0:1]) or (
            diff_neg_x<diff_pos_x and world_Coordinates2[0,0:1] >= world_Coordinates2[idx,0:1]):
            #print("move to last")
            vote += 1

        if (diff_neg_y > diff_pos_y and world_Coordinates2[0,1:2] >= world_Coordinates2[idx,1:2]) or (
            diff_neg_y < diff_pos_y and world_Coordinates2[0,1:2] < world_Coordinates2[idx,1:2]):
            #print("no need to move")
            vote += 0
        elif (diff_neg_y > diff_pos_y and world_Coordinates2[0,1:2] < world_Coordinates2[idx,1:2]) or (
            diff_neg_y < diff_pos_y and world_Coordinates2[0,1:2] >= world_Coordinates2[idx,1:2]):
            #print("move to last")
            vote += 1

        if (diff_neg_z > diff_pos_z and world_Coordinates2[0,2:3] >= world_Coordinates2[idx,2:3]) or (
            diff_neg_z < diff_pos_z and world_Coordinates2[0,2:3] < world_Coordinates2[idx,2:3]):
            #print("no need to move")
            vote += 0
        elif (diff_neg_z > diff_pos_z and world_Coordinates2[0,2:3] < world_Coordinates2[idx,2:3]) or (
            diff_neg_z < diff_pos_z and world_Coordinates2[0,2:3] >= world_Coordinates2[idx,2:3]):
            #print("move to last")
            vote += 1
        #print("vote:",vote)

        if vote >= 2:
            #Coordinates = np.roll(Coordinates, -1, axis=0)
            #world_Coordinates2 = np.roll(world_Coordinates2, -1, axis=0)
            Coordinates = np.delete(Coordinates, 0, axis=0)
            world_Coordinates2 = np.delete(world_Coordinates2, 0, axis=0)

            ctr+=1
            vote=0
        else:
            if not silent:
                print("number of rolls:",ctr)
            break


#    while True:
#        if np.count_nonzero( (world_Coordinates2[0][:3] >= world_Coordinates2[idx][:3]) ) < 2:
#            Coordinates = np.roll(Coordinates, -1, axis=0)
#            world_Coordinates2 = np.roll(world_Coordinates2, -1, axis=0)
#            ctr+=1
#        else:
#            if not silent:
#                print("number of rolls:",ctr)
#            break


            
    world_Coordinates2, idx_wc = np.unique(world_Coordinates2, axis=0, return_index=True)
    world_Coordinates2 = world_Coordinates2[np.argsort(idx_wc)]
    
    Coordinates, idx_wc = np.unique(Coordinates, axis=0, return_index=True)
    Coordinates = Coordinates[np.argsort(idx_wc)]
    
    return Coordinates, world_Coordinates2

def Publish_coordinates(Coordinates, parent_name, child_name, static = False): #Coordinates expects [[0,0,0,0,0,0,0],[1,1,1,1,1,1,1]] format

    for index in range (len(Coordinates)):
        
        if static==True:
            static_t.header.stamp = rospy.Time.now()
            static_t.header.frame_id = parent_name #"camera_depth_optical_frame"
            static_t.child_frame_id = child_name+"_static_"+str(index)
            static_t.transform.translation.x = Coordinates[index][0]
            static_t.transform.translation.y = Coordinates[index][1]
            static_t.transform.translation.z = Coordinates[index][2]

            #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)
            static_t.transform.rotation.x = Coordinates[index][3]
            static_t.transform.rotation.y = Coordinates[index][4]
            static_t.transform.rotation.z = Coordinates[index][5]
            static_t.transform.rotation.w = Coordinates[index][6]
            static_br.sendTransform(static_t)
        else:
            t.header.frame_id = parent_name #"camera_depth_optical_frame"
            t.child_frame_id = child_name+"_"+str(index)

            t.header.stamp = rospy.Time.now()
            t.transform.translation.x = Coordinates[index][0]
            t.transform.translation.y = Coordinates[index][1]
            t.transform.translation.z = Coordinates[index][2] 

            #r_quat = tf.transformations.quaternion_from_euler(Roll,Pitch,Yaw)
            t.transform.rotation.x = Coordinates[index][3]
            t.transform.rotation.y = Coordinates[index][4]
            t.transform.rotation.z = Coordinates[index][5]
            t.transform.rotation.w = Coordinates[index][6]
            br.sendTransform(t)



def Cluster_Point_Cloud(original_PC, eps=0.02, min_points=10):

    labels = np.array(original_PC.cluster_dbscan(eps=eps, min_points=min_points))
    uniques = np.unique(labels)
    clouds = np.array(original_PC)
    
    if ((uniques[0] == -1) and (len(uniques) == 1)):
        clouds = np.append(clouds,[original_PC])

    else:
        for i in range (len(uniques)):

            if uniques[i] > -1:
                idx = np.where(labels==uniques[i])[0]
                cluster_pcd = original_PC.select_by_index(idx, invert=False)
                clouds = np.append(clouds,[cluster_pcd])

    clouds = np.delete(clouds, 0)
    print("Number of clusters:", len(clouds))
    return clouds


def go_to_coord_goal(move_group,coord):
    pose_goal = geometry_msgs.msg.Pose()
    pose_goal.position.x = coord[0]
    pose_goal.position.y = coord[1]
    pose_goal.position.z = coord[2]
    pose_goal.orientation.x = coord[3]
    pose_goal.orientation.y = coord[4]
    pose_goal.orientation.z = coord[5]
    pose_goal.orientation.w = coord[6]

    move_group.set_pose_target(pose_goal)
    
    success = move_group.go(wait=True)
    
    move_group.stop()
    move_group.clear_pose_targets()
    #current_pose = move_group.get_current_pose().pose

def go_to_joint_state(move_group,joint_goal):
    success = move_group.go(joint_goal, wait=True)
    move_group.stop()

def array_to_data(array):
    im = PIL_img.fromarray(array)
    output_buffer = BytesIO()
    im.save(output_buffer, format="PNG")
    data = output_buffer.getvalue()
    return data

def fetch_cloud_image(pointCloud, RX=0, RY=0, RZ=0):
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
    mesh_pc= mesh.sample_points_uniformly(number_of_points=1000000, use_triangle_normal=False)
    tmp_cloud = copy.deepcopy(pointCloud)  #To avoid overwriting the original point cloud
    
    tmp_cloud+=mesh_pc
    tmp_Rot = pointCloud.get_rotation_matrix_from_xyz((np.radians(RX), np.radians(RY), np.radians((RZ))))
    
    tmp_cloud.rotate(tmp_Rot, center=(0, 0, 0))
    
    vis = o3d.visualization.Visualizer() 
    vis.create_window(visible=False, width=640, height=480) 
    vis.add_geometry(tmp_cloud) 
    vis.poll_events() 
    vis.update_renderer() 
    color = vis.capture_screen_float_buffer(True) 
    #time.sleep(5)
    vis.destroy_window() 
    #color = np.asarray(color)
    color = (255.0 * np.asarray(color)).astype(np.uint8)
    color = array_to_data(color) #Format according to the GUI requirements
    return color

def Cluster_selection_gui(clouds):
    dat=[]  #to hold cluster's images
    pcd_combined = o3d.geometry.PointCloud()
    
    for current_cloud in clouds:
            color = fetch_cloud_image(current_cloud, RX=120,RZ=180)  #different point clouds to be viewed, set view by RX,RY,RZ
            dat.append(color)
    result = subprocess.run([sys.executable,  "Cluster_selection_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(dat))   
    OP = literal_eval(result.stdout)
    selected_PC = OP[0]
    selected_motion = OP[1]
    
    for k in selected_PC:
        pcd_combined += clouds[k]  #combine selected pointclouds 
    return pcd_combined, selected_motion

#def Convert_message(string):
#    return [int(s) for s in re.findall('[0-9]', string)]

def get_XY_angles_from_PC(tst_downpcd_nor):
    Xs = []
    Ys = []
    vec_x = [1.,0.,0.] 
    vec_y = [0.,1.,0.]
    
    for normal_vec in tst_downpcd_nor:
        if np.all(normal_vec==[0., 0., 1.]): normal_vec = [0, 0.000001, 0.999999]
        vec1 = normal_vec / np.linalg.norm(normal_vec)
        angle_x = np.round(np.degrees((math.acos(np.dot(vec1, vec_x)))))
        angle_y = np.round(np.degrees((math.acos(np.dot(vec1, vec_y)))))
        Xs.append(angle_x)
        Ys.append(angle_y)
    Xs = np.array(Xs)
    Ys = np.array(Ys)
    return Xs, Ys


def Generate_final_coordinates(world_coords, z_offset, eef_link, coord_skip=3):

    Cam_target_final_coordinates=[]
    EEF_target_final_coordinates=[]

    for id_x in range(0,len(world_coords),coord_skip):  #Take every 4th point in coordinates
        transform_world_plane = world_coords[id_x]
        KDL_original_plane_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_world_plane[3],
                                                                         transform_world_plane[4], 
                                                                         transform_world_plane[5], 
                                                                         transform_world_plane[6]),
                                               PyKDL.Vector(transform_world_plane[0], 
                                                            transform_world_plane[1], 
                                                            transform_world_plane[2]))

        trans_x,trans_y,trans_z = KDL_original_plane_frame * PyKDL.Vector(0, 0, z_offset) #Add offset
        KDL_original_plane_frame.p = PyKDL.Vector(trans_x,trans_y,trans_z) #update original plane frame to new location

        KDL_flip_frame = PyKDL.Frame(PyKDL.Rotation.RPY(np.pi, 0.,np.pi), PyKDL.Vector(0, 0, 0)) #mirror plane frame to match camera frame

        KDL_final_frame = KDL_original_plane_frame * KDL_flip_frame

        KDL_trans = KDL_final_frame.p
        KDL_ROT_quat = KDL_final_frame.M.GetQuaternion() 

        final_coordinates = [KDL_trans[0], KDL_trans[1], KDL_trans[2], KDL_ROT_quat[0], KDL_ROT_quat[1], 
                                                                       KDL_ROT_quat[2], KDL_ROT_quat[3]]
        #print(final_coordinates)
        #Publish_coordinates([final_coordinates], "world", 'Camera_Target', static = False)

        Cam_target_final_coordinates.append(final_coordinates)

        #Fetch transform between eef link (link8, or tcp) and optical depth cam
        transform_eef_camera_depth = fetch_transform(tfbuffer,'camera_depth_optical_frame', eef_link, quat=1)

        KDL_eef_cam_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_eef_camera_depth[3],
                                                                    transform_eef_camera_depth[4], 
                                                                    transform_eef_camera_depth[5],
                                                                    transform_eef_camera_depth[6]),
                                          PyKDL.Vector(transform_eef_camera_depth[0], 
                                                       transform_eef_camera_depth[1], 
                                                       transform_eef_camera_depth[2])) #eef-camera frame

        #Offset original frame by z offset, flip the frame to match camera, multiply with eef-camera frame to 
        #replicate pose between camera and eef
        KDL_final_frame = KDL_original_plane_frame * KDL_flip_frame * KDL_eef_cam_frame 

        KDL_trans = KDL_final_frame.p
        KDL_ROT_quat = KDL_final_frame.M.GetQuaternion() 

        final_coordinates = [KDL_trans[0], KDL_trans[1], KDL_trans[2], KDL_ROT_quat[0], KDL_ROT_quat[1], 
                                                                       KDL_ROT_quat[2], KDL_ROT_quat[3]]
        #Publish_coordinates([final_coordinates], "world", 'EEF_Target', static = True)
        EEF_target_final_coordinates.append(final_coordinates)
        
    #Cam_target_final_coordinates.append(Cam_target_final_coordinates.pop(0))  #first element to last
    #EEF_target_final_coordinates.append(EEF_target_final_coordinates.pop(0))  #first element to last
    #Cam_target_final_coordinates.pop(0)
    #EEF_target_final_coordinates.pop(0)
    return Cam_target_final_coordinates, EEF_target_final_coordinates

def filter_generate_coordinates(result, spacing, sample = 3, silent=True, remove_close_targets=True): 
    #This will sample generated coordinates and compare to find the ones with anomalies and discard them
    CC = []
    WC = []
    dist =[]
    if sample < 3:
        print("Warning! min samples required: 3, seting to 3.")
        sample=3
    for i in range(0,sample):
        cam_coords, world_coords = generate_coordinates(result,silent=False)

        median = np.median(world_coords[:,:3],axis=0)
        distance = np.mean(np.linalg.norm(median - world_coords[:,:3], axis=1))
        CC.append(cam_coords)
        WC.append(world_coords)
        dist.append(distance)
    selection = np.argwhere(abs(np.ediff1d(dist)) < np.median(abs(np.ediff1d(dist))))[0][0]
    cam_T = CC[selection]
    WC_T = WC[selection]
    
    if remove_close_targets==True:
        i = 0
        while True:
            #print(i, len(WC_T))
            dis = spatial.distance.euclidean(WC_T[i-1][:3], WC_T[i][:3])
            #print(i-1,i, dis)
            if dis < 2*spacing:
                WC_T = np.delete(WC_T, i,axis=0)
                cam_T = np.delete(cam_T, i,axis=0)
            else:
                i+=1
            if i >= len(WC_T):
                break       
    
    return cam_T, WC_T

def Front_gui(data_in=0):
    result = subprocess.run([sys.executable,  "Front_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(data_in))
    data_out = literal_eval(result.stdout)
    return data_out

def Setting_gui(data_in):

    result = subprocess.run([sys.executable,  "Settings_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr(data_in))
    data_out = literal_eval(result.stdout)
    return data_out

def Manual_gui(manual_offset, TGT_save):
    
    dat=[]  #to hold images
    for f in sorted(glob.iglob("VI_appdata/Robo_object_positions/*")):
        im = PIL_img.open(f)
        output_buffer = BytesIO()
        im.save(output_buffer, format="PNG")
        data = output_buffer.getvalue()
        dat.append(data)
   
    result = subprocess.run([sys.executable,  "Manual_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr([dat,manual_offset, TGT_save]))   

    OP = literal_eval(result.stdout)
    return OP[0], OP[1], OP[2],OP[3] #pose_idx, man_offset, TGT_save, Exit_flag

def Auto_gui(manual_offset, TGT_save):
    
    dat=[]  #to hold images
    for f in sorted(glob.iglob("VI_appdata/Robo_object_positions/*")):
        im = PIL_img.open(f)
        output_buffer = BytesIO()
        im.save(output_buffer, format="PNG")
        data = output_buffer.getvalue()
        dat.append(data)
   
    result = subprocess.run([sys.executable,  "Auto_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr([dat,manual_offset, TGT_save]))   

    OP = literal_eval(result.stdout)
     #selected_mode, man_offset, cluster_idx, selected_motion, TGT_save, iterations,Hide_prev,Exit_flag
    return OP[0], OP[1], OP[2], OP[3], OP[4], OP[5], OP[6], OP[7]


def Replay_gui(file_path, X_offset, Y_offset, Z_offset, TGT_save):
      
    result = subprocess.run([sys.executable,  "Replay_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr([file_path, X_offset, Y_offset, Z_offset, TGT_save]))   

    OP = literal_eval(result.stdout)
    return OP[0], OP[1], OP[2], OP[3], OP[4], OP[5] #file_path, X_offset, Y_offset, Z_offset, TGT_save, OK_flag


def Save_gui():
      
    result = subprocess.run([sys.executable,  "Save_gui.py"],capture_output=True,text=True,check=True,shell=False, input=repr([0]))   
    OP = literal_eval(result.stdout)
    return OP[0], OP[1] #file_path, Save_mode


def get_average_pt_cloud(offset_y,offset_z,trim_base,Dbug):
    zoom_def = 1.34
    cam_robo_depth = abs(fetch_transform(tfbuffer,'camera_depth_optical_frame', 'panda_link0',quat=0)[2]) #get z value
    color_frame, depth_frame = grab_frame()
    downpcd = Generate_PointCloud(color_frame,depth_frame,trim_base,from_depth=False, 
                                  depth_trunc = cam_robo_depth - trim_base, align=False,Dbug=Dbug) #Take from 4cm above ground
    
    #CROP TO FILTER OUT ROBOT'S SHADOW ADJUST OFFSET ACCORDINGLY
    PC_BBOX = downpcd.get_axis_aligned_bounding_box()
    minB_X = PC_BBOX.min_bound[0]
    maxB_X = PC_BBOX.max_bound[0]
    minB_Y = PC_BBOX.min_bound[1]
    maxB_Y = PC_BBOX.max_bound[1]
    minB_Z = PC_BBOX.min_bound[2]
    maxB_Z = PC_BBOX.max_bound[2]

    bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(minB_X, minB_Y, minB_Z), 
                                               max_bound=(maxB_X, maxB_Y-offset_y, maxB_Z-offset_z)) 
    downpcd = downpcd.crop(bbox)

    if Dbug==True:
        mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
        o3d.visualization.draw_geometries([downpcd,mesh], window_name="02_Filtered Pointcloud: Remove Ground and Robot", point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)

    return downpcd


def Load_PC(samples,offset_y,offset_z,spacing,trim_base,Hide_prev=False,Dbug=False):
    zoom_def = 1.34
    front_def = [ -0.38077889171563734, 0.78942164088651434, -0.48147783804018851 ]
    lookat_def = [ 0.16328584993371112, 0.10901604638723184, 0.39735867391549773 ]
    up_def = [ 0.17393478184042019, -0.45025906577602964, -0.87579304938588232 ]

    if (Hide_prev==False or Dbug==True):
        mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1,origin=[0, 0, 0])
    
    
    average_ptcloud = o3d.geometry.PointCloud()

    pts_cloud_tot = []
    pts_tot = []

    for i in range(samples):
        new_pt = get_average_pt_cloud(offset_y,offset_z,trim_base,Dbug)
        #average_ptcloud += new_pt
        pts_cloud_tot.append(new_pt)
        pts_tot.append(len(np.asarray(new_pt.points)))


    #Collected Point clouds
    if Dbug==True:
        snip = pts_cloud_tot
        snip.append(mesh)
        o3d.visualization.draw_geometries(snip, window_name="03_Sampled Pointclouds", point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)


    ## We'll select only those point clouds that have a majority of similar "number of points". 
    ## this will prevent any outlier point clouds that have weird data in it (like less points 
    ## or more points than the group)

    bins = np.linspace(np.min(pts_tot), np.max(pts_tot), int(np.ceil(samples/3)))  #generate bins in range of number of pts in ptcloud
    digitized = np.digitize(pts_tot, bins) #put values in bins based on size of point clouds

    uni, count = np.unique(digitized,return_counts=True) #get the uniques and their counts
    unique_val = uni[np.argmax(count)]  #get the value from unique list where count was maximum
    idx_list = np.where(digitized == unique_val)[0]  #get all indexes from digitized list where it matches unique

    for idx in idx_list:
        average_ptcloud += pts_cloud_tot[idx]

    #Average pointcloud having majority of similar point clouds
    if Dbug==True:
        o3d.visualization.draw_geometries([average_ptcloud,mesh], window_name="04_Filtered Pointcloud: Select valid pointclouds by majority vote", point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)


    ## Filter out hidden points
    diameter = np.linalg.norm(np.asarray(average_ptcloud.get_max_bound()) - np.asarray(average_ptcloud.get_min_bound()))
    cam = [0, 0, -diameter]  #[0, 0, diameter]
    radius = diameter * 100
    _, pt_map = average_ptcloud.hidden_point_removal(cam, radius) #Get all points that are visible from given view point
    average_ptcloud = average_ptcloud.select_by_index(pt_map)

    #Hidden point Removal
    if Dbug==True:
        o3d.visualization.draw_geometries([average_ptcloud,mesh], window_name="05_Filtered Pointcloud: Hidden point removal", point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)

    average_ptcloud = average_ptcloud.voxel_down_sample(voxel_size=spacing)

    #Downsample point cloud
    if Dbug==True:
        o3d.visualization.draw_geometries([average_ptcloud,mesh], window_name="06_Filtered Pointcloud: Downsample the pointcloud", point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)

    #Estimate Normals
    average_ptcloud.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.06, max_nn=30)) #radius in meters
    average_ptcloud.orient_normals_towards_camera_location(camera_location=[0, 0, 0])

    #Re = average_ptcloud.get_rotation_matrix_from_xyz((np.pi, 0 , 0))
    #filtered_Pt_Cloud = average_ptcloud.rotate(Re, center=(0,0,0))

    filtered_Pt_Cloud = average_ptcloud

    if (Hide_prev==False or Dbug==True):
        o3d.visualization.draw_geometries([filtered_Pt_Cloud,mesh], window_name="07_Filtered Pointcloud: Estimate surface normals", point_show_normal=True, zoom=zoom_def, front=front_def, lookat=lookat_def, up=up_def)

    return(filtered_Pt_Cloud)


def New_world_coordinates(world_coords, eef_link, new_z_offset=0.0, old_z_offset=0.0,  coord_skip=1):
    
    z_offset = new_z_offset#old_z_offset+new_z_offset

    new_world_coordinates_store=[]

    for id_x in range(0,len(world_coords),coord_skip):
        transform_world_plane = world_coords[id_x]
        KDL_original_plane_frame = PyKDL.Frame(PyKDL.Rotation.Quaternion(transform_world_plane[3],
                                                                         transform_world_plane[4], 
                                                                         transform_world_plane[5], 
                                                                         transform_world_plane[6]),
                                               PyKDL.Vector(transform_world_plane[0], 
                                                            transform_world_plane[1], 
                                                            transform_world_plane[2]))

        trans_x,trans_y,trans_z = KDL_original_plane_frame * PyKDL.Vector(0, 0, z_offset) #Add offset
        KDL_original_plane_frame.p = PyKDL.Vector(trans_x,trans_y,trans_z) #update original plane frame to new location

        #KDL_flip_frame = PyKDL.Frame(PyKDL.Rotation.RPY(np.pi, 0.,np.pi), PyKDL.Vector(0, 0, 0)) #not required here
        KDL_final_frame = KDL_original_plane_frame #* KDL_flip_frame
        KDL_trans = KDL_final_frame.p
        KDL_ROT_quat = KDL_final_frame.M.GetQuaternion() 
        final_coordinates = [KDL_trans[0], KDL_trans[1], KDL_trans[2], KDL_ROT_quat[0], KDL_ROT_quat[1], 
                                                                       KDL_ROT_quat[2], KDL_ROT_quat[3]]

        new_world_coordinates_store.append(final_coordinates)
    #world_coords = np.append(world_coords, np.array(new_world_coordinates),axis=0)
    
    return new_world_coordinates_store


print("Loaded!!")

In [ ]:
front_def = [ -0.38077889171563734, 0.78942164088651434, -0.48147783804018851 ]
lookat_def = [ 0.16328584993371112, 0.10901604638723184, 0.39735867391549773 ]
up_def = [ 0.17393478184042019, -0.45025906577602964, -0.87579304938588232 ]
saved_path = "VI_appdata/Saved_coordinates/"

pose_list = ['initial_coordinates_down_med', 'initial_coordinates_down_high', 'initial_coordinates_front_low',
             'initial_coordinates_front_med', 'initial_coordinates_left_origin_low', 'initial_coordinates_left_origin_med',
             'initial_coordinates_left_extended_low', 'initial_coordinates_left_extended_med', 'initial_coordinates_right_origin_low',
             'initial_coordinates_right_origin_med', 'initial_coordinates_right_extended_low', 'initial_coordinates_right_extended_med']

pose_list_joint = ['initial_coordinates_down_med_joint', 'initial_coordinates_down_high_joint', 'initial_coordinates_front_low_joint',
                   'initial_coordinates_front_med_joint', 'initial_coordinates_left_origin_low_joint', 'initial_coordinates_left_origin_med_joint',
                   'initial_coordinates_left_extended_low_joint', 'initial_coordinates_left_extended_med_joint', 'initial_coordinates_right_origin_low_joint',
                   'initial_coordinates_right_origin_med_joint', 'initial_coordinates_right_extended_low_joint','initial_coordinates_right_extended_med_joint']

config = ConfigParser() #recreate the object
file_path = 'VI_appdata/'
file_name = 'config.ini'

while True:
    if len(config.read(file_path+file_name)) < 1:
        print("File not Found! Creating file with default parameters.")

        config.add_section('Settings')
        config.set('Settings', 'samples', '1')
        config.set('Settings', 'spacing', '0.01')
        config.set('Settings', 'offset_y', '0.13')
        config.set('Settings', 'offset_z', '0.0')
        config.set('Settings', 'trim_base', '0.05')
        config.set('Settings', 'manual_offset', '0.0')
        config.set('Settings', 'Cluster_centered', 'True')
        config.set('Settings', 'Cluster_idx', '0')
        config.set('Settings', 'Cluster_discard', '0')
        config.set('Settings', 'eps', '0.05')
        config.set('Settings', 'min_points', '10')
        config.set('Settings', 'Cluster_trim', '0.01')
        config.set('Settings', 'TGT_coord_Samples', '3')
        config.set('Settings', 'TGT_final_trim', '0.8')
        config.set('Settings', 'TGT_reverse', 'True')
        config.set('Settings', 'TGT_preview', 'True')
        config.set('Settings', 'z_offset', '0.3')
        config.set('Settings', 'coord_skip', '0')
        config.set('Settings', 'TGT_motion_delay', '0.1')
        config.set('Settings', 'TGT_save', 'True')
        config.set('Settings', 'Dbug', 'False')
        
        
        config.add_section('Init_Pose')
        config.set('Init_Pose', 'initial_coordinates_front_low', str([0.397980905857042, 0.027217939895356813, 0.18229966557867178, 
                             0.719466879923975, -0.024833366401559, 0.6938126371395357, 0.019358128812197253]))
        config.set('Init_Pose', 'initial_coordinates_front_low_joint', str([-0.03308832927721994, -0.03018116998842668, 0.2330358247909432, 
                           -3.067160927725503, -2.877206619976536, 1.7093383191263651, -2.3699487627687947]))

        config.set('Init_Pose', 'initial_coordinates_front_med', str([0.40482032529532164, 0.026194852526629283, 0.39109200963985485, 
                     0.7260836395583325, -0.022024302679397548, 0.6869470017604175, 0.020525477572626547]))
        config.set('Init_Pose', 'initial_coordinates_front_med_joint', str([-2.85595247676397, 0.8446004103871667, 2.8969771730651814, 
                       -2.935194399235205, -2.7707237618641196, 2.598331237598268, -2.8520272715670796]))
        
        config.set('Init_Pose', 'initial_coordinates_down_med', str([0.3057885688125996, 1.3924967197681087e-05, 0.4832398935268236, 
                   0.9999795762494257, 0.0001837540182692045, -0.0063807165029802415, 0.0003158724238107869]))
        config.set('Init_Pose', 'initial_coordinates_down_med_joint', str([0.00015612302097078867, -0.7820471788872219, 0.00021312723955091428, 
                     -2.3608742714695437, -0.00048321686683028275, 1.5660659497089506, 0.7853428543922201]))
        
        config.set('Init_Pose', 'initial_coordinates_down_high', str([0.4073103269798378,-1.4228208118709305e-05,0.7305284183578298,
                       0.9999928346346129,-0.00048259177856538034,-0.003741879327895847,0.00031003822734821697]))
        config.set('Init_Pose', 'initial_coordinates_down_high_joint', str([-0.0004353885679213576, -0.09060550395190248, 0.0006182921638773209, 
                       -1.0642032332881817, -0.0006107609171950301, 0.9526048542598877, 0.7866203388740391]))
        
        config.set('Init_Pose', 'initial_coordinates_right_extended_low', str([0.303905123995592, 0.432574462138798, 0.14745536781503069, 
                        0.49793866097971806, -0.5190453811419719, 0.5022512997938495, 0.4799923062636517]))
        config.set('Init_Pose', 'initial_coordinates_right_extended_low_joint', str([-1.3627551805652836, -1.665272628644563, 1.8213776531030774, 
                      -0.9998667790998539, 0.23454549403950065, 0.8435528353614163, 2.7865978631457864]))
        
        config.set('Init_Pose', 'initial_coordinates_right_extended_med', str([0.2946072656492903, 0.42412609694782166, 0.3812023180158996, 
                      0.5004812310890651, -0.5145842693664336, 0.500097519321071, 0.48438005555255736]))
        config.set('Init_Pose', 'initial_coordinates_right_extended_med_joint', str([-1.2149762059636426, -1.4147605703375774, 1.630446034507517, 
                        -1.220881668558492, -0.12494140705733248, 0.8627764897864765, 2.414921812055187]))
        
        config.set('Init_Pose', 'initial_coordinates_right_origin_low', str([0.304617061088216, 0.010824979104373134, 0.13103384627481848, 
                            0.5000567302260833, -0.5176586342440617, 0.5011957042311564, 0.4803911645819538]))
        config.set('Init_Pose', 'initial_coordinates_right_origin_low_joint', str([-0.8306507813405739, -1.6702893075232446, 2.0899260568853126, 
                          -2.583962115788799, 0.4729478447971518, 1.9204627252998598, 2.5709161872287396]))
        
        config.set('Init_Pose', 'initial_coordinates_right_origin_med', str([0.2950873959916672, 0.0022689810690858375, 0.35247625585972037, 
                        0.5000080875838175, -0.517773630947132, 0.4994838572729616, 0.4820977657802621]))
        config.set('Init_Pose', 'initial_coordinates_right_origin_med_joint', str([-0.7394367859002831, -1.7502600075394135, 1.545832156050504, 
                      -2.7715569703412393, 0.1468177821017873, 1.9215750098383229, 2.189953048092672]))
        
        config.set('Init_Pose', 'initial_coordinates_left_extended_low', str([0.31155232781852343, -0.4086689597065088, 0.13700900414256567, 
                 -0.5010520177929303, -0.50991813124275, -0.48424832821784847, 0.5044144441961924]))
        config.set('Init_Pose', 'initial_coordinates_left_extended_low_joint', str([1.4792665598288135, -1.3786874126536794, -2.2556703813351007, 
                       -1.1207441668097236, 0.06761726972025528, 0.9208591206339669, -1.436089059953261]))
        
        config.set('Init_Pose', 'initial_coordinates_left_extended_med', str([0.31388297259319375, -0.4034255830492924, 0.3812481778815888, 
                     -0.5007903951112147, -0.5087092508711633, -0.48435410833506043, 0.5057914352590425]))
        config.set('Init_Pose', 'initial_coordinates_left_extended_med_joint', str([1.3489994479032257, -1.0385404174043078, -2.0826207583573364, 
                           -1.2522584530530603, 0.4164006949380532, 0.807743989710545, -1.132394918608119]))
        
        config.set('Init_Pose', 'initial_coordinates_left_origin_low', str([0.30019161622355806, -0.008566905843574349, 0.13068973171451095, 
                   -0.5008108668868128, -0.5113460029542676, -0.4841846837677913, 0.5032682514081904]))
        config.set('Init_Pose', 'initial_coordinates_left_origin_low_joint', str([1.1857468951962424, -1.0613242297651801, -2.0859292543370067, 
                     -2.593607189178292, 0.18790388228584032, 1.9826879664409294, -1.5356077559260912]))
        
        config.set('Init_Pose', 'initial_coordinates_left_origin_med', str([0.3022296225047282, -0.0016561879639682836, 0.3924491069637494, 
                   -0.5001942516935315, -0.511437879038984, -0.4833357967164821, 0.5046023326039006]))
        config.set('Init_Pose', 'initial_coordinates_left_origin_med_joint', str([0.6729239245732437, -1.070449681957646, -1.2528592387187087, 
                 -2.6905143631133868, 0.5376461765737943, 1.757936868031745, -1.0937173345324362]))

        # save to a file
        with open(file_path+file_name, 'w') as configfile:
            config.write(configfile)
        
        config = ConfigParser() #recreate the object
        config.read(file_path+file_name) #Read the created file
    else:
        #Load all variables
        samples = max(config.getint('Settings', 'samples'),1)
        spacing = max(config.getfloat('Settings', 'spacing'),0)
        offset_y = config.getfloat('Settings', 'offset_y')
        offset_z = config.getfloat('Settings', 'offset_z')
        trim_base = config.getfloat('Settings', 'trim_base')
        manual_offset = config.getfloat('Settings', 'manual_offset')
        Cluster_centered = config.getboolean('Settings', 'Cluster_centered')
        Cluster_idx = config.getint('Settings', 'Cluster_idx')
        Cluster_discard = config.getint('Settings', 'Cluster_discard')
        eps = config.getfloat('Settings', 'eps')
        min_points = config.getint('Settings', 'min_points')
        Cluster_trim = config.getfloat('Settings', 'Cluster_trim')
        TGT_coord_Samples = max(config.getint('Settings', 'TGT_coord_Samples'),3)
        TGT_final_trim = config.getfloat('Settings', 'TGT_final_trim')
        TGT_reverse = config.getboolean('Settings', 'TGT_reverse')
        TGT_preview = config.getboolean('Settings', 'TGT_preview')
        z_offset = config.getfloat('Settings', 'z_offset')
        coord_skip = max(config.getint('Settings', 'coord_skip'),0)+1 #+1 to match the for loop's skip method.
        TGT_motion_delay = config.getfloat('Settings', 'TGT_motion_delay')
        TGT_save = config.getboolean('Settings', 'TGT_save')
        Dbug = config.getboolean('Settings', 'Dbug')

        init_data = [samples,spacing,offset_y,offset_z,trim_base,manual_offset,Cluster_centered,Cluster_idx,
                 Cluster_discard,eps,min_points,Cluster_trim,TGT_coord_Samples,TGT_final_trim,TGT_reverse,
                 TGT_preview,z_offset,coord_skip-1,TGT_motion_delay,TGT_save,Dbug] #-1 to display real meaning in settings gui

        Front_data = Front_gui() # Call Front end GUI to select running mode.
        
        if Front_data == -1:
            print("Exiting Program!")
            break

#REPLAY MODE =====================================================================================================            
#REPLAY MODE =====================================================================================================            
        elif Front_data == 0:
            print("Run in Replay mode")
            file_path='/'
            X_offset=0.0 
            Y_offset=0.0
            Z_offset=0.0
            
            while True:
                file_path, X_offset, Y_offset, Z_offset, TGT_save, OK_flag = Replay_gui(file_path, X_offset, Y_offset, Z_offset, TGT_save)
                
                if OK_flag == 0: #Preview
                    print("Load coordinates, Add offsets and publish to Rviz and return to GUI")
                    
                    with open(file_path, 'rb') as f:
                        world_coords = np.load(f)
                    del f

                    world_coords[:,0:1] += X_offset
                    world_coords[:,1:2] += Y_offset
                    
                    eef_link = move_group.get_end_effector_link()
                    cam_tgt, eef_tgt = Generate_final_coordinates(world_coords, Z_offset+z_offset, eef_link, coord_skip=1)
                    Publish_coordinates(cam_tgt, "world", 'Camera_Target', static = False)
                    #Publish_coordinates(eef_tgt, "world", 'EEF_Target', static = False)
                    #Publish_coordinates(world_coords,'world','plane', static = False)

                    
                elif OK_flag == 1: #OK
                    print("Load coordinates, Add offsets and publish to Rviz and run and break out")
                    
                    with open(file_path, 'rb') as f:
                        world_coords = np.load(f)
                    del f

                    world_coords[:,0:1] += X_offset
                    world_coords[:,1:2] += Y_offset
                    
                    
                    eef_link = move_group.get_end_effector_link()
                    cam_tgt, eef_tgt = Generate_final_coordinates(world_coords, Z_offset+z_offset, eef_link, coord_skip=1)

                    if TGT_preview == True:
                        Publish_coordinates(cam_tgt, "world", 'Camera_Target', static = False)
                        #Publish_coordinates(eef_tgt, "world", 'EEF_Target', static = False)

                    for id_x in range(0,len(eef_tgt)):  

                        Publish_coordinates([cam_tgt[id_x]], "world", 'Camera_Target', static = False)   
                        #Publish_coordinates([eef_tgt[id_x]], "world", 'EEF_Target', static = False)

                        ## Move Robot to TARGET
                        go_to_coord_goal(move_group, eef_tgt[id_x]) #pass values to function to make robot move in cartesian space. 
                        print("Moving to Target:",id_x+1)
                        time.sleep(TGT_motion_delay)


                    world_coords_new = New_world_coordinates(world_coords, eef_link, new_z_offset=Z_offset, old_z_offset=z_offset, coord_skip=1)
                    
                    #Save Coordinates
                    if TGT_save == True:
                        print("Open Save Dialogue")

                        file_path, Save_mode = Save_gui()
                        if Save_mode == -1:
                            break
                        elif Save_mode == 0: #Create_New_file
                            print("Creating New file")
                            if len(sorted(glob.iglob(saved_path+"saved_*"))) > 0:
                                last_file = sorted(glob.iglob(saved_path+"saved_*"))[-1].strip('.npy')
                                file_num = int(last_file[len(last_file) - 3:])+1 #new file number
                                file_num_str = "{0:0=3d}".format(file_num)
                                coord_filename = "saved_" + file_num_str
                            else:
                                coord_filename = "saved_000"
                            coord_path = saved_path+coord_filename+".npy"

                            with open(coord_path, 'wb') as f:
                                np.save(f, world_coords_new)

                            break

                        elif Save_mode == 1: #Overwrite default saved file of manual mode
                            print("Overwriting saved.npy file")

                            coord_filename = "saved"
                            coord_path = saved_path+coord_filename+".npy"

                            with open(coord_path, 'wb') as f:
                                np.save(f, world_coords_new)

                            break

                        elif Save_mode == 2: #Append coordinates to selected file
                            print("Append to selected file")

                            #Load old coordinates
                            with open(file_path, 'rb') as f:
                                old_world_coords = np.load(f)
                                
                            del f
                            
                            #Append New coordinates to old coordinates
                            world_coords_save = np.append(old_world_coords, world_coords_new, axis=0)
                            print(world_coords_save)
                            #Save coordinates to same file
                            with open(file_path, 'wb') as g:
                                np.save(g, world_coords_save)                    

                            break

                    
                elif OK_flag == -1: #EXIT
                    print("Exit Replay mode")
                    break
                
            break

#MANUAL MODE =====================================================================================================
#MANUAL MODE =====================================================================================================
        elif Front_data == 1:
            print("Run in Manual mode")  #Move robot to a specific pose.                        
            pose_idx, manual_offset, TGT_save,Exit_flag = Manual_gui(manual_offset, TGT_save) # Call Manual mode GUI.
            
            if Exit_flag==1:
                print("Exiting Manual mode")
                Front_data = -1
                break
            
            
            #selected_pose = eval(config.get('Init_Pose', pose_list[pose_idx])) #eval will revert char string to whatever it was.
            #go_to_coord_goal(move_group, selected_pose) #pass values to function to make robot move in cartesian space. 
            selected_pose_joint = eval(config.get('Init_Pose', pose_list_joint[pose_idx])) #eval will revert char string to whatever it was.
            print("Moving to initial position")
            go_to_joint_state(move_group, selected_pose_joint)
            
            #Fetch filtered PC.
            filtered_Pt_Cloud = Load_PC(samples,offset_y,offset_z,spacing,trim_base,Hide_prev=False,Dbug=Dbug)  
            
            #Clustering
            clouds = Cluster_Point_Cloud(filtered_Pt_Cloud, eps=eps, min_points=min_points) 

            result,selected_motion = Cluster_selection_gui(clouds) #Will present a GUI to select all relevant pointclouds
            #o3d.visualization.draw_geometries([result, mesh], point_show_normal=True)

            result = cropped_PC(result, spacing, Cluster_trim, X=selected_motion, idx = Cluster_idx, centered = Cluster_centered, resample = True)
            if (Hide_prev==False or Dbug==True):
                zoom_def = 2.0
                mesh_big = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3,origin=[0, 0, 0])
                o3d.visualization.draw_geometries([result,mesh_big], window_name="08_Filtered Pointcloud: Object profile generated", point_show_normal=True, zoom=zoom_def, 
                                                  front=front_def, lookat=lookat_def, up=up_def)
                       
            #Coordinate Generation
            _, world_coords = filter_generate_coordinates(result, spacing, sample=TGT_coord_Samples, silent=False, remove_close_targets=True)
            print("Total coordinates:",len(world_coords))
            if TGT_reverse == True:
                world_coords = world_coords[::-1]  #reverse

            eef_link = move_group.get_end_effector_link()  # get current eef link either link8 if panda_arm selected 
                                                           # or hand_tcp if panda_manipulator is selected.
            cam_tgt, eef_tgt = Generate_final_coordinates(world_coords, z_offset, eef_link, coord_skip=coord_skip)

            #Coordinate PREVIEW
            if TGT_preview == True:
                Publish_coordinates(cam_tgt, "world", 'Camera_Target', static = False)
                #Publish_coordinates(eef_tgt, "world", 'EEF_Target', static = False)

            #ROBOT MOTION
            for id_x in range(0,len(eef_tgt)):  

                Publish_coordinates([cam_tgt[id_x]], "world", 'Camera_Target', static = False)   
                #Publish_coordinates([eef_tgt[id_x]], "world", 'EEF_Target', static = False)

                ## Move Robot to TARGET
                go_to_coord_goal(move_group, eef_tgt[id_x]) #pass values to function to make robot move in cartesian space. 
                print("Moving to Target:",id_x+1)
                time.sleep(TGT_motion_delay)

            #Save Coordinates
            if TGT_save == True:
                print("Open Save Dialogue")
                
                file_path, Save_mode = Save_gui()
                if Save_mode == -1:
                    break
                elif Save_mode == 0: #Create_New_file
                    print("Creating New file")
                    if len(sorted(glob.iglob(saved_path+"saved_manual_*"))) > 0:
                        last_file = sorted(glob.iglob(saved_path+"saved_manual_*"))[-1].strip('.npy')
                        file_num = int(last_file[len(last_file) - 3:])+1 #new file number
                        file_num_str = "{0:0=3d}".format(file_num)
                        coord_filename = "saved_manual_" + file_num_str
                    else:
                        coord_filename = "saved_manual_000"
                    coord_path = saved_path+coord_filename+".npy"
                    
                    with open(coord_path, 'wb') as f:
                        np.save(f, world_coords)
                        
                    break
                    
                elif Save_mode == 1: #Overwrite default saved file of manual mode
                    print("Overwriting saved_manual.npy file")
                    
                    coord_filename = "saved_manual"
                    coord_path = saved_path+coord_filename+".npy"

                    with open(coord_path, 'wb') as f:
                        np.save(f, world_coords)
                    
                    
                    break
                
                elif Save_mode == 2: #Append coordinates to selected file
                    print("Append to selected file")
                    
                    #Load old coordinates
                    with open(file_path, 'rb') as f:
                        old_world_coords = np.load(f)
                        
                    #Append New coordinates to old coordinates
                    world_coords_save = np.append(old_world_coords, world_coords,axis=0)
                    
                    #Save coordinates to same file
                    with open(file_path, 'wb') as f:
                        np.save(f, world_coords_save)                    
                        
                        
                    break
                
            break

#AUTO MODE =====================================================================================================
#AUTO MODE =====================================================================================================
        elif Front_data == 2:
            print("Run in Auto mode")
            world_coords_total = np.array([[0.,0.,0.,0.,0.,0.,0.]])
            last = []

            pose_idx, manual_offset, cluster_number, selected_motion, TGT_save, Auto_loop, Hide_prev,Exit_flag = Auto_gui(manual_offset, TGT_save)

            if Exit_flag==1:
                print("Exiting Auto mode")
                Front_data = -1
                break
            #selected_pose = eval(config.get('Init_Pose', pose_list[pose_idx])) #eval will revert char string to whatever it was.
            #go_to_coord_goal(move_group, selected_pose) #pass values to function to make robot move in cartesian space. 
            selected_pose_joint = eval(config.get('Init_Pose', pose_list_joint[pose_idx])) #eval will revert char string to whatever it was.
            print("Moving to initial position")
            go_to_joint_state(move_group, selected_pose_joint)
            
            
            for i in range(Auto_loop):
            
                filtered_Pt_Cloud = Load_PC(samples,offset_y,offset_z,spacing,trim_base,Hide_prev)  #Fetch filtered PC.
            
                #Clustering
                clouds = Cluster_Point_Cloud(filtered_Pt_Cloud, eps=eps, min_points=min_points) 
                
                if Hide_prev == False:
                    result,selected_motion = Cluster_selection_gui(clouds) #Will present a GUI to select all relevant pointclouds
                    #o3d.visualization.draw_geometries([result, mesh], point_show_normal=True)
                    result = cropped_PC(result, spacing, Cluster_trim, X=selected_motion, idx = Cluster_idx, centered = Cluster_centered, resample = True)
                else:
                    result = cropped_PC(clouds[cluster_number], spacing, Cluster_trim, X=selected_motion, idx = Cluster_idx, centered = Cluster_centered, resample = True)
                
                if (Hide_prev==False or Dbug==True):
                    zoom_def = 2.0
                    mesh_big = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.3,origin=[0, 0, 0])
                    o3d.visualization.draw_geometries([result,mesh_big], window_name="08_Filtered Pointcloud: Object profile generated", point_show_normal=True, zoom=zoom_def, 
                                                          front=front_def, lookat=lookat_def, up=up_def)
                        

                _, world_coords = filter_generate_coordinates(result, spacing, sample=TGT_coord_Samples, silent=False, remove_close_targets=True)
                print("Total coordinates:",len(world_coords))
                
                if TGT_reverse == True:
                    world_coords = world_coords[::-1]  #reverse
                    
                if len(last)>0:  #if list is not empty
                    idx = np.argmin(np.linalg.norm(last - world_coords[:,:3], axis=1)) #give the index of value where it is most similar
                    world_coords = world_coords[idx:]

                last = world_coords[-1,:3]

                world_coords_total = np.append(world_coords_total,world_coords,axis=0)


                eef_link = move_group.get_end_effector_link()
                cam_tgt, eef_tgt = Generate_final_coordinates(world_coords, z_offset, eef_link, coord_skip=coord_skip)


                if TGT_preview == True:
                    Publish_coordinates(cam_tgt, "world", 'Camera_Target', static = False)
                    #Publish_coordinates(eef_tgt, "world", 'EEF_Target', static = False)


                for id_x in range(0,len(eef_tgt)):  

                    Publish_coordinates([cam_tgt[id_x]], "world", 'Camera_Target', static = False)   
                    #Publish_coordinates([eef_tgt[id_x]], "world", 'EEF_Target', static = False)

                    ## Move Robot to TARGET
                    go_to_coord_goal(move_group, eef_tgt[id_x]) #pass values to function to make robot move in cartesian space. 
                    print("Moving to Target:",id_x+1)
                    time.sleep(TGT_motion_delay)

            world_coords_total = np.delete(world_coords_total, 0, axis=0)
            

            #Save Coordinates
            if TGT_save == True:
                print("Open Save Dialogue")
                
                file_path, Save_mode = Save_gui()
                if Save_mode == -1:
                    print("Not Saving")
                    break
                elif Save_mode == 0: #Create_New_file
                    print("Creating New file")
                    if len(sorted(glob.iglob(saved_path+"saved_Auto_*"))) > 0:
                        last_file = sorted(glob.iglob(saved_path+"saved_Auto_*"))[-1].strip('.npy')
                        file_num = int(last_file[len(last_file) - 3:])+1 #new file number
                        file_num_str = "{0:0=3d}".format(file_num)
                        coord_filename = "saved_Auto_" + file_num_str
                    else:
                        coord_filename = "saved_Auto_000"
                    coord_path = saved_path+coord_filename+".npy"
                    
                    with open(coord_path, 'wb') as f:
                        np.save(f, world_coords_total)
                        
                    break
                    
                elif Save_mode == 1: #Overwrite default saved file of manual mode
                    print("Overwriting saved_Auto.npy file")
                    
                    coord_filename = "saved_Auto"
                    coord_path = saved_path+coord_filename+".npy"

                    with open(coord_path, 'wb') as f:
                        np.save(f, world_coords_total)
                    
                    
                    break
                
                elif Save_mode == 2: #Append coordinates to selected file
                    print("Append to selected file")
                    
                    #Load old coordinates
                    with open(file_path, 'rb') as f:
                        old_world_coords = np.load(f)
                        
                    #Append New coordinates to old coordinates
                    world_coords_save = np.append(old_world_coords, world_coords_total,axis=0)
                    
                    #Save coordinates to same file
                    with open(file_path, 'wb') as f:
                        np.save(f, world_coords_save)                    
                        
                    break
                
            break


#SETTINGS MODE =====================================================================================================
#SETTINGS MODE =====================================================================================================
        elif Front_data == 3:
            #Open settings
            Settings_data = Setting_gui(init_data)

            status_flag = Settings_data[0]

            if status_flag == 1:  #update values and save new config as string

                config.set('Settings', 'samples', str(Settings_data[1]))
                config.set('Settings', 'spacing', str(Settings_data[2]))
                config.set('Settings', 'offset_y', str(Settings_data[3]))
                config.set('Settings', 'offset_z', str(Settings_data[4]))
                config.set('Settings', 'trim_base', str(Settings_data[5]))
                config.set('Settings', 'manual_offset', str(Settings_data[6]))
                config.set('Settings', 'Cluster_centered', str(Settings_data[7]))
                config.set('Settings', 'Cluster_idx', str(Settings_data[8]))
                config.set('Settings', 'Cluster_discard', str(Settings_data[9]))
                config.set('Settings', 'eps', str(Settings_data[10]))
                config.set('Settings', 'min_points', str(Settings_data[11]))
                config.set('Settings', 'Cluster_trim', str(Settings_data[12]))
                config.set('Settings', 'TGT_coord_Samples', str(Settings_data[13]))
                config.set('Settings', 'TGT_final_trim', str(Settings_data[14]))
                config.set('Settings', 'TGT_reverse', str(Settings_data[15]))
                config.set('Settings', 'TGT_preview', str(Settings_data[16]))
                config.set('Settings', 'z_offset', str(Settings_data[17]))
                config.set('Settings', 'coord_skip', str(Settings_data[18]))
                config.set('Settings', 'TGT_motion_delay', str(Settings_data[19]))
                config.set('Settings', 'TGT_save', str(Settings_data[20]))
                config.set('Settings', 'Dbug', str(Settings_data[21]))

                # save to a file
                with open(file_path+file_name, 'w') as configfile:
                    config.write(configfile)

#print(samples,spacing,offset_y,offset_z,trim_base,manual_offset,Cluster_centered,Cluster_idx,
#      Cluster_discard,eps,min_points,Cluster_trim,TGT_coord_Samples,TGT_final_trim,TGT_reverse,
#      TGT_preview,z_offset,coord_skip,TGT_motion_delay,TGT_save,Dbug)


In [ ]:
#move_group.get_current_pose()
#go_to_joint_state(move_group, selected_pose_joint)

### Clustering of points in Point cloud
ref: http://www.open3d.org/docs/release/tutorial/geometry/pointcloud.html#DBSCAN-clustering

### Crop point cloud to get a points in a single line in X or Y axis, 
*Use idx to give offset else use centered to get the points from center of the object*

### This will generate and boradcast all points in pointcloud to tf2 or tf2_static topic, visualize using Rviz

## Publish Targets for Preview

## Robot Motion